## Chargement des données

In [ ]:
import pandas as pd
# On charge le fichier csv qui se trouve dans un sous-dossier 'data'
data = pd.read_csv("./data/GlobalTemperatures.csv")
# On charge les températures moyennes (terre et océan confondus) dans une série de données Pandas,
# à partir de la 1300ème ligne (il n'y a aucune donnée avant cela).
# On réalise aussi une interpolation pour remplir automatiquement certaines valeurs manquantes
temperature = data["LandAndOceanAverageTemperature"].interpolate(limit_direction="backward")[1300:]
dates = pd.to_datetime(data["dt"])[1300:]

### Stationnarisation

In [ ]:
from statsmodels.tsa.stattools import adfuller

def make_stationary(time_series):
    """
    Makes a time series stationary while the p-value computed using the ADF test is higher than 0.05
    """
    test_results = adfuller(time_series)
    p_value = test_results[1]
    if p_value > 0.05: # i.e. if the data is not stationary
        # Differentiating until the p-value goes under 0.05
        diff_data = time_series.copy()
        for degree in range(1, 10):
            diff_data = diff_data.diff().dropna()
            if adfuller(diff_data)[1] <= 0.05:
                return diff_data, degree
        
        raise ValueError(f"Unable to stationarize data after diferentiating {degree} times")
    return time_series

_, diff_degree = make_stationary(temperature)
print(f"Degrees of differentiating: {diff_degree}")

### Préparation des datasets d'entraînement et de test, recherche des meilleurs hyperparamètres pour ARIMA

In [ ]:
from pmdarima.model_selection import train_test_split

# On commence par séparer nos jeux d'entraînement et de test
training_samples_count = int(len(dates) * 0.9)
train, test = train_test_split(temperature, train_size=training_samples_count)
train_dates, future_dates = train_test_split(dates, train_size=training_samples_count)

In [ ]:
# Recherche des meilleurs paramètres pour notre ARIMA
# Comme nous avons déjà déterminé que notre degré de différenciation
# est de 1, nous pouvons le préciser à auto_arima pour gagner du temps
# d'exécution
from pmdarima import auto_arima
# On choisit une valeur de 12 pour la saisonnalité car nos données
# sont des moyennes mensuelles
best_model = auto_arima(train, d=1, seasonal=True, m=12, stepwise=False, n_jobs=-1, trace=True)
print(best_model)

### Entraînement

In [ ]:
from pmdarima.arima import ARIMA

# On instancie notre modèle avec les hyperparamètres définis plus tôt
model = ARIMA(
    order=(1, 1, 1),
    seasonal_order=(2, 0, 1, 12)
               )
model.fit(train)

predictions_count = len(test)
# On réalise des prédictions à comparer avec les valeurs de test
forecast, confidence_intervals = model.predict(predictions_count, return_conf_int=True) 


### Visualisation des résultats

In [ ]:
# Adding confidence intervals
ci_dict = {
    "low": confidence_intervals[:, 0],
    "high": confidence_intervals[:, 1]
}
confidence_intervals_df = pd.DataFrame(ci_dict)
confidence_intervals_df.index = future_dates

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(train_dates) + len(future_dates))
trend = np.polyfit(x, pd.concat([train, forecast]), 1)


trendpoly = np.poly1d(trend)
plt.plot(dates, temperature, c='blue')
plt.plot(future_dates, forecast, c='green')
plt.plot(dates, trendpoly(x), c="red")
plt.fill_between(confidence_intervals_df.index, confidence_intervals_df['low'], confidence_intervals_df['high'], alpha=0.9, color='orange', label="Confidence intervals")
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(15,8))



# Computing yearly average in order to increase readibility
temperature.index = dates
yearly_temperature = temperature.rolling(window=12, step=12, center=False).mean().dropna()

train.index = train_dates
yearly_train = train.rolling(window=12, step=12, center=False).mean().dropna()

forecast.index = future_dates
yearly_forecast = forecast.rolling(window=12, step=12, center=False).mean().dropna()

train_and_forecast = pd.concat([yearly_train, yearly_forecast])
x = np.arange(len(train_and_forecast))
trend = np.polyfit(x, train_and_forecast, 1)
trendpoly = np.poly1d(trend)

yearly_train_dates = pd.Series(yearly_train.index)
yearly_forecast_dates = pd.Series(yearly_forecast.index)
all_dates = pd.concat([yearly_train_dates, yearly_forecast_dates])

yearly_confidence_intervals_df = confidence_intervals_df.rolling(window=12, step=12, center=False).mean().dropna()

plt.plot(yearly_temperature, c='blue', label='Températures annuelles réelles')
plt.plot(yearly_forecast_dates, yearly_forecast.values, c='green', label='Températures annuelles prédites')
#plt.plot(train_and_forecast.index, trendpoly(x), c="red", label='Forecast trend')
plt.fill_between(yearly_confidence_intervals_df.index, yearly_confidence_intervals_df['low'], yearly_confidence_intervals_df['high'], alpha=0.9, color='orange', label="Intervalles de confiance")
plt.legend(loc='upper left')
plt.gcf().autofmt_xdate()


### Calcul de la RMSE

In [ ]:
# Calcul de la RMSE
import math
from sklearn.metrics import mean_squared_error

def rmse(true_values, predicted_values):
    """
    Computes the Root Mean-Squared Error of the predicted values with regards to the actual values.
    """
    return math.sqrt(mean_squared_error(true_values, predicted_values))

# Exemple avec nos prédictions :
print(f"RMSE : {rmse(test, forecast)}")
